# Wealth Scenerio Simulator
The goal of this is to get the different scenarios of capital allocations (holding in bank, holdng crypto, holding stocks) in regards to different scenerios such as:
- Paying the house off and saving money to live off of
- Paying the house off then investing (With different ratios)
- Paying good chunk of interest off on house and start investing
- Start investing right away pay regular payments on house


In [33]:
pip install numpy-financial


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\btindol\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [34]:
import pandas as pd
import numpy as np
from datetime import date
import numpy_financial as npf


In [9]:
start_date = date.today() # starting today which happens to be the first 

years = 30 # 30 years

payments_year = 12 # 12 payments a year..so 12 periods

# Gives you a start date that increases monthly for the next 30 years... 
rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
rng

DatetimeIndex(['2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01',
               ...
               '2050-07-01', '2050-08-01', '2050-09-01', '2050-10-01',
               '2050-11-01', '2050-12-01', '2051-01-01', '2051-02-01',
               '2051-03-01', '2051-04-01'],
              dtype='datetime64[ns]', length=360, freq='MS')

In [89]:
# Makes a date range from today to number of years yous specify chunking into months..
######################################################################################
######################################################################################
# Parameter making
# year parameters
start_date = date.today() # starting today which happens to be the first 
Years = 30 # 30 years
payment_periods_in_year = 12 # 12 payments a year..so 12 periods
rng = pd.date_range(start_date, periods=years * payment_periods_in_year, freq='MS')

Total_Mortgage_Cost_At_Start = 142000
Interest_Rate = 0.02875
Payment_Periods_In_Year = 12 #
Additional_Mortgage_Principal_Added = 50

MonthlyIncome = 6000
Expenses = 3000
StockAnnualReturn =10  # for 10 %
StockMonthlyReturnMonthly = (StockAnnualReturn/12)/100 # annual return divided by 12 is .88 and .88 is really .0088 so divide by 100
CryptoMonthlyReturn =.10
DividendYeild = .03 # 3% yeild of dividiends
#####################################################################################
######################################################################################
# Index creation
df = pd.DataFrame(index=rng,columns=['Monthly_Income','Expenses','Bank_Balance_Post_Expenses','Bank_Account_Balance', 'Mortgage_Total_Payment', 'Mortgage_Principal_Payment','Mortgage_Interest_Payment','Additional_Mortgage_Principal_Added','Cumulative_Mortgage_Principal_Paid','Cumulative_Mortgage_Interest_Paid','Current_Mortgage_Balance_Left_To_Pay','Dividend_Stock_Additional_Return_Monthly_100%','Monthly_Dividends_100_Current_Total_Balance_100%','Dividends_This_Month_100%_Invested','Crypto_Stock_Additional_Return_Monthly_100%','Monthly_Crypto_100_Current_Total_Balance_100%'], dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"
######################################################################################
######################################################################################

df["Monthly_Income"] =MonthlyIncome
df["Expenses"] = Expenses
df["Bank_Balance_Post_Expenses"] = df["Monthly_Income"] -df["Expenses"]
df["Bank_Account_Balance"] = df["Bank_Balance_Post_Expenses"].cumsum()

df["Mortgage_Total_Payment"] = round(npf.pmt(Interest_Rate/Payment_Periods_In_Year, Years*Payment_Periods_In_Year, Total_Mortgage_Cost_At_Start),2)
df["Mortgage_Principal_Payment"] = npf.ppmt(Interest_Rate/Payment_Periods_In_Year, df.index, Years*Payment_Periods_In_Year, Total_Mortgage_Cost_At_Start)
df["Mortgage_Interest_Payment"] = npf.ipmt(Interest_Rate/Payment_Periods_In_Year, df.index, Years*Payment_Periods_In_Year, Total_Mortgage_Cost_At_Start)
df["Additional_Mortgage_Principal_Added"] = -Additional_Mortgage_Principal_Added

#Store the Cumulative Principal Payments and ensure it never gets larger than the original principal
df["Cumulative_Mortgage_Principal_Paid"] = (df["Mortgage_Principal_Payment"] + df["Additional_Mortgage_Principal_Added"]).cumsum()
df["Cumulative_Mortgage_Principal_Paid"] = round(df["Cumulative_Mortgage_Principal_Paid"].clip(lower=-Total_Mortgage_Cost_At_Start),2)
df["Cumulative_Mortgage_Interest_Paid"] = round(df["Mortgage_Interest_Payment"].cumsum(),2)
df["Current_Mortgage_Balance_Left_To_Pay"] = Total_Mortgage_Cost_At_Start + df["Cumulative_Mortgage_Principal_Paid"]

# Mortgage appriciation rate??????

# Making the different percentage allocations for dividend stocks for return of 3% 

df['Dividend_Stock_Additional_Return_Monthly_100%'] = df['Bank_Account_Balance'] * StockMonthlyReturnMonthly
df['Monthly_Dividends_100_Current_Total_Balance_100%'] =  df['Dividend_Stock_Additional_Return_Monthly_100%'] + df['Bank_Account_Balance']
df['Dividends_This_Month_100%_Invested'] = round((df['Monthly_Dividends_100_Current_Total_Balance_100%'] * DividendYeild)/12,2)
df['Dividend_Stock_Additional_Return_Monthly_100%'] = df['Bank_Account_Balance'] * StockMonthlyReturnMonthly


# Making the different percentage allocations for crypto allocation 
df['Crypto_Stock_Additional_Return_Monthly_100%'] = df['Bank_Account_Balance'] * CryptoMonthlyReturn
df['Monthly_Crypto_100_Current_Total_Balance_100%'] =  df['Crypto_Stock_Additional_Return_Monthly_100%'] + df['Bank_Account_Balance']

#Crypto_Balance_100%
df





,index,Monthly_Income,Expenses,Bank_Balance_Post_Expenses,Bank_Account_Balance,Mortgage_Total_Payment,Mortgage_Principal_Payment,Mortgage_Interest_Payment,Additional_Mortgage_Principal_Added,Cumulative_Mortgage_Principal_Paid,Cumulative_Mortgage_Interest_Paid,Current_Mortgage_Balance_Left_To_Pay,Dividend_Stock_Additional_Return_Monthly_100%,Monthly_Dividends_100_Current_Total_Balance_100%,Dividends_This_Month_100%_Invested,Crypto_Stock_Additional_Return_Monthly_100%,Monthly_Crypto_100_Current_Total_Balance_100%
Period,,,,,,,,,,,,,,,,,
1,2021-05-01,6000,3000,3000,3000,-589.15,-248.938984,-340.208333,-50,-298.94,-340.21,141701.06,25.0,3025.0,7.56,300.0,3300.0
2,2021-06-01,6000,3000,3000,6000,-589.15,-249.535400,-339.611917,-50,-598.47,-679.82,141401.53,50.0,6050.0,15.12,600.0,6600.0
3,2021-07-01,6000,3000,3000,9000,-589.15,-250.133246,-339.014072,-50,-898.61,-1018.83,141101.39,75.0,9075.0,22.69,900.0,9900.0
4,2021-08-01,6000,3000,3000,12000,-589.15,-250.732523,-338.414794,-50,-1199.34,-1357.25,140800.66,100.0,12100.0,30.25,1200.0,13200.0
5,2021-09-01,6000,3000,3000,15000,-589.15,-251.333237,-337.814081,-50,-1500.67,-1695.06,140499.33,125.0,15125.0,37.81,1500.0,16500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2050-12-01,6000,3000,3000,1068000,-589.15,-582.140267,-7.007050,-50,-142000.00,-70078.99,0.00,8900.0,1076900.0,2692.25,106800.0,1174800.0
357,2051-01-01,6000,3000,3000,1071000,-589.15,-583.534978,-5.612339,-50,-142000.00,-70084.60,0.00,8925.0,1079925.0,2699.81,107100.0,1178100.0
358,2051-02-01,6000,3000,3000,1074000,-589.15,-584.933031,-4.214287,-50,-142000.00,-70088.81,0.00,8950.0,1082950.0,2707.38,107400.0,1181400.0
